In [7]:
# import ipynb
# import torch
# from ipynb.fs.full.data_handler import load_subject, load_subject_non_downsampled, extract_labels, extract_action_interval, split_data, split_info, to_device, get_innerspeech
# from ipynb.fs.full.train_model import train_model, accuracy_check

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# data, description = load_subject_non_downsampled(1)
# data, description = get_innerspeech(data, description)
# labels = extract_labels(description)
# #data_interval = extract_action_interval(data, hz = 254)

# train_data, val_data, test_data, train_labels, val_labels, test_labels = split_data(data, labels)

# train_data, val_data, test_data, train_labels, val_labels, test_labels = to_device(train_data, val_data, test_data, train_labels, val_labels, test_labels, device)
# split_info(train_data, val_data, test_data, train_labels, val_labels, test_labels)

[39, 39, 40, 40] 
 [6, 6, 5, 5] 
 [5, 5, 5, 5]
24.68354430379747 %  24.68354430379747 %  25.31645569620253 %  25.31645569620253 % 
27.27272727272727 %  27.27272727272727 %  22.727272727272727 %  22.727272727272727 % 
25.0 %  25.0 %  25.0 %  25.0 % 


# Deep CNN
Implementation of the deep CNN structure from (Schirrmeister et. al.) for the use on "Thinking out loud" dataset.

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

class DeepCNN(nn.Module):
    def __init__(self, hz, interval = "full", dropout = 0.5):
        super(DeepCNN, self).__init__()
        first_channels = 25
        second_channels = 50
        third_channels = 100
        fourth_channels = 200

        ### Block 1
        # Temporal convolution
        self.tempconv = nn.Conv2d(in_channels = 1, out_channels = first_channels, kernel_size = (1, 10), padding = 0, bias = False)
        # Spatial convolution
        self.spatconv = nn.Conv2d(in_channels = first_channels, out_channels = first_channels, kernel_size=(128,1), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm1 = nn.BatchNorm2d(first_channels, False)
        # ELU
        self.elu = nn.ELU()
        # Dropout
        self.dropout = nn.Dropout(dropout)
        # Max pooling
        self.maxpool = nn.MaxPool2d(kernel_size = (1,3), stride = (1,3))

        ### Block 2
        self.conv2 = nn.Conv2d(in_channels = first_channels, out_channels = second_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm2 = nn.BatchNorm2d(second_channels, False)

        ### Block 3
        self.conv3 = nn.Conv2d(in_channels = second_channels, out_channels = third_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm3 = nn.BatchNorm2d(third_channels, False)

        ### Block 4
        self.conv4 = nn.Conv2d(in_channels = third_channels, out_channels = fourth_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm4 = nn.BatchNorm2d(fourth_channels, False)

        # Classifier
        if interval == "action":
            if hz == 254:
                self.classifier = nn.Linear(600,4, bias = False)
            elif hz == 1024:
                self.classifier = nn.Linear(5400,4, bias = False)
        elif interval == "full":
            if hz == 254:
                self.classifier = nn.Linear(1800,4, bias = False)
            elif hz == 1024:
                self.classifier = nn.Linear(10400,4, bias = False)
        # Softmax
        self.softmax = nn.LogSoftmax(dim = 1)

        
    def forward(self, x):
        # Block 1
        res = self.tempconv(x)
        res = self.spatconv(res)
        res = self.batchnorm1(res)
        res = self.elu(res)
        res = self.maxpool(res)
        res = self.dropout(res)
        # Block 2
        res = self.conv2(res)
        res = self.batchnorm2(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Block 3
        res = self.conv3(res)
        res = self.batchnorm3(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Block 4
        res = self.conv4(res)
        res = self.batchnorm4(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Classifier
        res = torch.flatten(res, start_dim=1)
        res = self.classifier(res)
        res = self.softmax(res)
        return res

In [9]:
# network = DeepCNN(interval = "full", hz = 1024).float().to(device)
# accuracy_check(network, train_data, train_labels)

# print("####### TRAINING #######")
# op = optim.Adam(params = network.parameters(), lr = 0.0001)
# lossf = nn.NLLLoss()
# train_model(network, train_data = train_data, train_labels = train_labels, val_data = val_data, val_labels = val_labels, epochs = 50, batch_size = 10, loss_func = lossf, optimizer = op)

# accuracy_check(network, train_data, train_labels)

ACCURACY: 0.25949367088607594
####### TRAINING #######
Epoch	 train loss	 validation loss
0 	  1.646026865641276 	 1.5778151750564575
1 	  1.2198935190836588 	 1.9929646253585815
2 	  0.9084199905395508 	 2.2632029056549072
3 	  0.6502815246582031 	 2.2950401306152344
4 	  0.5513851165771484 	 2.361945390701294
5 	  0.4171006202697754 	 2.506869316101074
6 	  0.3230495770772298 	 2.6426260471343994
7 	  0.25980539321899415 	 2.8152365684509277
8 	  0.21368227005004883 	 2.873865842819214
9 	  0.17674469947814941 	 2.9069101810455322
10 	  0.1372761885325114 	 2.9336814880371094
11 	  0.1146597941716512 	 3.0292868614196777
12 	  0.10223600069681803 	 3.104403018951416
13 	  0.0893762747446696 	 3.156200647354126
14 	  0.07502603530883789 	 2.90048885345459
15 	  0.06130613883336385 	 2.6328070163726807
16 	  0.0607585072517395 	 2.498098134994507
17 	  0.052350294589996335 	 2.441368579864502
18 	  0.04688220818837484 	 2.5335984230041504
19 	  0.04193726380666097 	 2.5232157707214355


In [11]:
# accuracy_check(network, train_data, train_labels)
# accuracy_check(network, val_data, val_labels)
# accuracy_check(network, test_data, test_labels)


ACCURACY: 0.930379746835443
ACCURACY: 0.2727272727272727
ACCURACY: 0.1
